In [3]:
import glob
import pandas as pd
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Tools.exp_tools import Experiment


def get_df(path):
    df = pd.read_csv(path)
    df.rename(
        columns={'GlobalID': 'droplet_id', 'Effector': 'effector', 'Target': 'target', 'dead_Effector': 'dead_effector',
                 'dead_Target': 'dead_target'}, inplace=True)
    df = df.astype({'droplet_id': int})
    df.set_index('droplet_id', inplace=True)
    df.drop(columns=['i'], inplace=True)
    return df

In [4]:
exp_id = 'NKIP_FA_082'
experiment = Experiment(exp_id)

In [ ]:
# add droplets to db

droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id

experiment.db.add_droplets(droplets)
tfrecord_manifest = {
    'n_frames': droplets.index.size,
    'y_shape': 128,
    'x_shape': 128,
    'n_channels': experiment.handler.channel_df.index.size,
    'channel_info': experiment.handler.channel_df['channel_name'].to_dict(),
}
experiment.db.update_manifest(tfrecord_manifest)

In [ ]:
experiment.db.clear_annotations()

In [ ]:
# add cell_count predictions to annotations table
prefix = 'cell_count_v3'
droplets = pd.read_csv(os.path.join(os.getenv('EXP_DIR'), exp_id, 'droplets.csv'))
droplets.rename(columns={'GlobalID': 'droplet_id', 'frameID':'frame_id'}, inplace=True)
droplets['experiment_id'] = exp_id
droplets.set_index('droplet_id', inplace=True)
droplets.rename(columns={f'{prefix}_dead_Effector': 'dead_effector',
                         f'{prefix}_Target': 'target',
                         f'{prefix}_Effector': 'effector',
                         f'{prefix}_dead_Target': 'dead_target'}, inplace=True)
selected_annotations = ['dead_effector', 'dead_target', 'effector', 'target']
droplets[selected_annotations].head()

In [ ]:
source = 'cell_count_v3'
annotations = []

for droplet_id, droplet in droplets[selected_annotations].iterrows():
    for label_type, value  in droplet.items():
        annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'source': source})

annotation_df = pd.DataFrame(annotations)
print(annotation_df.head())
experiment.db.add_annotations(annotation_df)

In [ ]:
# Import workpackages
wps = glob.glob(os.path.join(os.getenv('EXP_DIR'), exp_id, '*', 'WP_*.csv'))
annotations = []
for wp in wps:
    wp_id = os.path.basename(wp)
    df = get_df(wp).sort_values('droplet_id')

    for droplet_id, droplet in df.iterrows():
        for label_type, value in droplet.items():
            if not pd.isna(value):
                annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': str(int(value)), 'ap_id': wp_id, 'status': 'completed'})
            else:
                annotations.append({'droplet_id': droplet_id, 'label_type': label_type, 'value': None, 'ap_id': wp_id, 'status': 'pending'})
    
    db_manifest = experiment.db.get_manifest()
    if 'annotation_packages' not in db_manifest.keys():
        db_manifest['annotation_packages'] = {}
    db_manifest['annotation_packages'][wp_id] = "None"
    experiment.db.update_manifest(db_manifest)

annotation_df = pd.DataFrame(annotations)
experiment.db.add_annotations(annotation_df)

